# Part 0: Raw Data & Environment Set-up

## Raw Data

The following code loads in and unzips the raw data. Raw data has been downloaded from Kaggle placed into public Google Cloud Storage for ease of access.

In [2]:
# Loads in steam-reviews.zip (game reviews)
url1 = ("https://storage.googleapis.com/dsc232r-group-project-data/steam-reviews.zip")
!wget "{url1}"

--2025-05-03 20:58:20--  https://storage.googleapis.com/dsc232r-group-project-data/steam-reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.91, 142.250.176.27, 142.250.189.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17241297932 (16G) [application/x-zip-compressed]
Saving to: ‘steam-reviews.zip’

steam-reviews.zip   100%[===================>]  16.06G   111MB/s    in 2m 25s  

2025-05-03 21:00:46 (113 MB/s) - ‘steam-reviews.zip’ saved [17241297932/17241297932]



In [3]:
# Extracts steam-reviews.zip into specified directory and deletes .zip file
!unzip steam-reviews.zip -d /home/joneel/joneel/Group_Project/raw_data/steam-reviews && rm steam-reviews.zip

Archive:  steam-reviews.zip
  inflating: /home/joneel/joneel/Group_Project/raw_data/steam-reviews/all_reviews/all_reviews.csv  
  inflating: /home/joneel/joneel/Group_Project/raw_data/steam-reviews/weighted_score_above_08.csv  


In [1]:
# Loads in games.csv (games metadata)
url2 = ("https://storage.googleapis.com/dsc232r-group-project-data/games.csv")
!wget "{url2}"

--2025-05-04 22:35:19--  https://storage.googleapis.com/dsc232r-group-project-data/games.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.217.155, 172.217.14.123, 142.250.68.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.217.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 468641107 (447M) [application/vnd.ms-excel]
Saving to: ‘games.csv’

games.csv           100%[===================>] 446.93M   115MB/s    in 4.3s    

2025-05-04 22:35:23 (105 MB/s) - ‘games.csv’ saved [468641107/468641107]



In [2]:
# Moves games.csv into specified directory
!mv games.csv /home/joneel/joneel/Group_Project/raw_data

## Environment Set-up

Set-up on the cluster included 30 cores with 60GB memory in order to load and process this dataset (total ~45GB).

In [1]:
# Import required libraries
import os, pickle, glob
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark import StorageLevel

In [2]:
#sc.stop() # To stop a currently running SparkSession, if needed for troubleshooting/development

In [3]:
# Establishes Spark Session
sc = SparkSession.builder \
    .config("spark.driver.memory", "2g") \
	.config("spark.executor.memory", "2g") \
    .config('spark.executor.instances', 69) \
	.appName("Review_Analysis") \
	.getOrCreate()

In [4]:
# Loads all_reviews.csv file into a spark dataframe
reviews_df = sc.read.csv("/home/joneel/joneel/Group_Project/raw_data/steam-reviews/all_reviews/all_reviews.csv", header=True, inferSchema=True)

In [6]:
# Displays reviews_df schema and counts total # of reviews
reviews_df.printSchema()
print(f"Number of reviews: {reviews_df.count()}")

root
 |-- recommendationid: string (nullable = true)
 |-- appid: string (nullable = true)
 |-- game: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: string (nullable = true)
 |-- author_playtime_forever: string (nullable = true)
 |-- author_playtime_last_two_weeks: string (nullable = true)
 |-- author_playtime_at_review: string (nullable = true)
 |-- author_last_played: string (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- voted_up: string (nullable = true)
 |-- votes_up: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- writt

In [7]:
# Removes two columns related to Chinese gaming market (not relevant for this analysis)
reviews_df = reviews_df.drop("hidden_in_steam_china", "steam_china_location")
reviews_df.printSchema()

root
 |-- recommendationid: string (nullable = true)
 |-- appid: string (nullable = true)
 |-- game: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: string (nullable = true)
 |-- author_playtime_forever: string (nullable = true)
 |-- author_playtime_last_two_weeks: string (nullable = true)
 |-- author_playtime_at_review: string (nullable = true)
 |-- author_last_played: string (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- voted_up: string (nullable = true)
 |-- votes_up: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- writt

In [8]:
# Filters dataframe to include only reviews in English & counts new number of reviews
reviews_df_processed = reviews_df.filter(reviews_df.language == 'english')
reviews_df_processed.select("language").distinct().show()
print(f"Number of reviews: {reviews_df_processed.count()}")

+--------+
|language|
+--------+
| english|
+--------+

Number of reviews: 51544179


In [46]:
# Drops rows that contain null values or duplicates in ids & review columns
reviews_df_processed = reviews_df_processed.na.drop(subset=["recommendationid", "appid", "author_steamid", "review"])
reviews_df_processed = reviews_df_processed.dropDuplicates(subset=["recommendationid"])
reviews_df_processed = reviews_df_processed.dropDuplicates(subset=["review"])
print(f"Number of reviews: {reviews_df_processed.count()}")

Number of reviews: 51544179


In [6]:
games_df = sc.read.csv("/home/joneel/joneel/Group_Project/raw_data/games.csv", header=True, inferSchema=True)

In [7]:
games_df.printSchema()
print(f"Number of games: {games_df.count()}")

root
 |-- appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- price: string (nullable = true)
 |-- dlc_count: double (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- about_the_game: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- website: string (nullable = true)
 |-- support_url: string (nullable = true)
 |-- support_email: string (nullable = true)
 |-- windows: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- metacritic_score: string (nullable = true)
 |-- metacritic_url: string (nullable = true)
 |-- achievements: string (nullable = true)
 |-- recommendations: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- full_audi

In [11]:
games_df_processed = games_df.drop("reviews", "header_image", "website", "support_url", "support_email", "full_audio_languages", "screenshots", "movies")
games_df_processed.printSchema()

root
 |-- appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- price: string (nullable = true)
 |-- dlc_count: double (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- about_the_game: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- windows: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- metacritic_score: string (nullable = true)
 |-- metacritic_url: string (nullable = true)
 |-- achievements: string (nullable = true)
 |-- recommendations: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- packages: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- user_score: str

In [12]:
print(f"Number of games: {games_df_processed.count()}")
games_df_processed = games_df_processed.na.drop(subset=["appid"])
print(f"Number of games: {games_df_processed.count()}")

Number of games: 89618
Number of games: 89618


In [ ]:
"Tokenize reviews, see # of reviews per author, break down tags and genres"
"Make some plots to see data distributions (# of reviews, reviews per author, dates, best and worst reviewed games?)"